In [60]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait, Select # no hara operaciones hasta q se carguen los elementos de la página
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os 
from datetime import date

In [56]:
def guarda_csv(df, titulo):
    fecha_data = date.today()
    nombre_archivo = fecha_data.strftime('%Y%m%d') + titulo + '.csv' 
    file_dir = os.path.dirname(os.path.abspath("_file_"))
    csv_folder = 'output/' + fecha_data.strftime('%Y') + "/" + fecha_data.strftime('%m')
    try:
        os.makedirs(os.path.join(file_dir, csv_folder))
    except FileExistsError:
        pass
    file_path = os.path.join(file_dir, csv_folder, nombre_archivo)
    df.to_csv(file_path, index=False)
    print('Archivo guardado', nombre_archivo)
    print("")

In [40]:
# Opciones de navegacion

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-extensions")

# Inicialimos el navegador 
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.lider.cl/supermercado/")




In [41]:
# Digitaremos la palabra azúcar y daremos click en buscar de forma automatica:
WebDriverWait(driver,5)\
    .until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input#searchtextinput")))\
    .send_keys("azucar")

WebDriverWait(driver,5)\
    .until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button#searchsubmitbutton")))\
    .click()

In [42]:
print(driver.current_url)

https://www.lider.cl/supermercado/search?Ntt=azucar&ost=azucar


In [43]:
#Funcion para capturar los objetos por etiqueta y clase
def buscar_tag_por_etiqueta_clase(html, etiqueta, clase):
    soup = BeautifulSoup(html, "html.parser")
    return soup.find_all(etiqueta, class_= re.compile(clase +"$"))
 

In [44]:
class Producto:
    pass

In [45]:
#usaremos request para consultar por una dirección url y obtener el contenido, en este caso leer productos de supermecado lider
# y en vez de llamar a la página WEb a través del dominio llamamos a través de driver.current 
ruta = driver.current_url
sitio = requests.get(ruta)

In [46]:
#Buscamos nuestros tag que contienen los productos del carro
listado_items = buscar_tag_por_etiqueta_clase(sitio.content, "div", "product-item-box")

In [47]:
#Ahora vamos a recorrer cada producto del carro, según la lista que encontramos en el paso anterior. Generamos una lista de productos donde 
#guardaremos nuestros objeto Producto, según los valores que rescatamos dentro del ciclo for

productos = []
for i in listado_items:
    producto = Producto()

        
    if (i.find('span', class_="product-description js-ellipsis", recursive=True) != []):
        producto.nombre = i.find('span', class_="product-description js-ellipsis", recursive=True).text
    
    if (i.find('span', class_="product-name", recursive=True) != []):
        producto.marca = i.find('span', class_="product-name", recursive=True).text
      
    
        
    if (i.find('span', class_="price-sell", recursive=True) != []):
        producto.precio = i.find('span', class_="price-sell", recursive=True).text

    producto.origen = "Lider"
    producto.fecha_captura = datetime.date.today().strftime("%Y-%m-%d")
    
    if (i.find('span', class_="product-attribute", recursive=True) != []):
        producto.formato = i.find('span', class_="product-attribute", recursive=True).text
        
    productos.append(producto)


In [48]:
#Creamos un dataframe a partir de nuestra lista de productos capturados en el paso anterior
df = pd.DataFrame.from_records([t.__dict__ for t in productos ])
df.head(200)

,nombre,marca,precio,origen,fecha_captura,formato
0,Azúcar Blanca,Iansa,$1.200,Lider,2022-09-06,1 Kg
1,Bebida Sin Azúcar,Coca-Cola,$2.110,Lider,2022-09-06,"2,5 L"
2,Bebida Sin Azúcar,Coca-Cola,$2.520,Lider,2022-09-06,3 L
3,Azúcar Blanca Granulada,Lider,$990,Lider,2022-09-06,1 kg
4,Bebida Sin Azúcar,Coca-Cola,$1.690,Lider,2022-09-06,"1,5 L"
5,Azúcar Blanca,Iansa,$5.990,Lider,2022-09-06,5 Kg
6,Bebida Sin Azúcar,Coca-Cola,$5.720,Lider,2022-09-06,"4 de 1,5 L c/u"
7,Merluza Filetes Con Piel,Lider,$3.990,Lider,2022-09-06,500 g
8,Néctar de Naranja Light 0% Azúcar Añadida,Watts,$1.240,Lider,2022-09-06,"1,5 L"
9,Cereal Sin Azúcar Añadida,Chocapic,$4.150,Lider,2022-09-06,720 g


In [50]:
df2 =df[df.marca.isin(["Lider","Iansa"])]
df3 =df2[df2["nombre"]!="Merluza Filetes Con Piel"]


In [51]:
# Ahora ordenamos el DataFrame por marca:
df3.sort_values("marca")

,nombre,marca,precio,origen,fecha_captura,formato
0,Azúcar Blanca,Iansa,$1.200,Lider,2022-09-06,1 Kg
5,Azúcar Blanca,Iansa,$5.990,Lider,2022-09-06,5 Kg
12,Azúcar Rubia,Iansa,$2.370,Lider,2022-09-06,1 Kg
20,Azúcar Rubia,Iansa,$1.080,Lider,2022-09-06,500 g
31,Azúcar Flor,Iansa,$1.090,Lider,2022-09-06,500 g
3,Azúcar Blanca Granulada,Lider,$990,Lider,2022-09-06,1 kg
14,Azúcar Granulada,Lider,$5.310,Lider,2022-09-06,5 kg


In [59]:
guarda_csv(df3,"lider")

NameError: name 'os' is not defined

In [13]:
# Web Scrapping página Tottus:

# Opciones de navegacion

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-extensions")

# Inicialimos el navegador 
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://tottus.falabella.com/tottus-cl")

In [14]:
# Digitaremos la palabra azúcar y daremos click en buscar de forma automatica:
WebDriverWait(driver,5)\
    .until(EC.element_to_be_clickable((By.CLASS_NAME,"SearchBar-module_searchBar__Input__1kPKS")))\
    .send_keys("azucar")

WebDriverWait(driver,5)\
    .until(EC.element_to_be_clickable((By.CLASS_NAME,"SearchBar-module_searchBtnIcon__2L2s0")))\
    .click()

In [15]:
print(driver.current_url)

https://tottus.falabella.com/tottus-cl/category/CATG10003/Azucar


In [16]:
# Web Scrapping página Tottus:

#Funcion para capturar los objetos por etiqueta y clase
def buscar_tag_por_etiqueta_clase(html, etiqueta, clase):
    soup = BeautifulSoup(html, "html.parser")
    return soup.find_all(etiqueta, class_= re.compile(clase +"$"))
 

In [17]:
class Producto:
    pass

In [20]:
#usaremos request para consultar por una dirección url y obtener el contenido, en este caso leer productos de supermecado Jumbo
ruta = driver.current_url
sitio = requests.get(ruta)



In [21]:
#Buscamos nuestros tag que contienen los productos del carro
listado_items_tottus = buscar_tag_por_etiqueta_clase(sitio.content, "div", "grid-pod")
print(len(listado_items_tottus))

12


In [22]:
#Ahora vamos a recorrer cada producto del carro, según la lista que encontramos en el paso anterior. Generamos una lista de productos donde 
#guardaremos nuestros objeto Producto, según los valores que rescatamos dentro del ciclo for

productos = []
for i in listado_items_tottus:
    producto = Producto()

        
    if (i.find('b', class_="subTitle-rebrand", recursive=True) != []):
        producto.nombre = i.find('b', class_="subTitle-rebrand", recursive=True).text
        
    if (i.find('span', class_="line-height-22", recursive=True) != []):
        producto.precio = i.find('span', class_="line-height-22", recursive=True).text
        
    if (i.find('b', class_="title-rebrand", recursive=True) != []):
        producto.marca = i.find('b', class_="title-rebrand", recursive=True).text
    
    
    producto.origen = "Tottus"
    producto.fecha_captura = datetime.date.today().strftime("%Y-%m-%d")
    productos.append(producto)

In [37]:
#Creamos un dataframe a partir de nuestra lista de productos capturados en el paso anterior
df = pd.DataFrame.from_records([t.__dict__ for t in productos ])
df.head(200)

,nombre,precio,marca,origen,fecha_captura
0,Azúcar - 1 kg,$ 890,TOTTUS,Tottus,2022-09-06
1,Azúcar Light Stevia - 500 g,$ 1.640,IANSA,Tottus,2022-09-06
2,Azúcar Flor - 1 k,$ 1.790,IANSA,Tottus,2022-09-06
3,Panela - 500 g,$ 2.390,TOTTUS,Tottus,2022-09-06
4,Chancaca - 450 g,$ 1.490,IANSA,Tottus,2022-09-06
5,Chancaca Deliciosa - 400 g,$ 2.110,Deliciosa,Tottus,2022-09-06
6,Azúcar Blanca Granulada - 1 kg,$ 1.390,IANSA,Tottus,2022-09-06
7,Panela granulada orgánica - un,$ 2.820,,Tottus,2022-09-06
8,Panela - 1 kg,$ 3.590,TOTTUS,Tottus,2022-09-06
9,Azúcar Integral - 750 g,$ 5.070,,Tottus,2022-09-06


In [38]:
df[["nombre","formato"]]= df["nombre"].str.split("-",expand=True)
df

,nombre,precio,marca,origen,fecha_captura,formato
0,Azúcar,$ 890,TOTTUS,Tottus,2022-09-06,1 kg
1,Azúcar Light Stevia,$ 1.640,IANSA,Tottus,2022-09-06,500 g
2,Azúcar Flor,$ 1.790,IANSA,Tottus,2022-09-06,1 k
3,Panela,$ 2.390,TOTTUS,Tottus,2022-09-06,500 g
4,Chancaca,$ 1.490,IANSA,Tottus,2022-09-06,450 g
5,Chancaca Deliciosa,$ 2.110,Deliciosa,Tottus,2022-09-06,400 g
6,Azúcar Blanca Granulada,$ 1.390,IANSA,Tottus,2022-09-06,1 kg
7,Panela granulada orgánica,$ 2.820,,Tottus,2022-09-06,un
8,Panela,$ 3.590,TOTTUS,Tottus,2022-09-06,1 kg
9,Azúcar Integral,$ 5.070,,Tottus,2022-09-06,750 g


In [52]:
def guarda_csv(df, titulo):
    fecha_data = date.today()
    nombre_archivo = fecha_data.strftime('%Y%m%d') + titulo + '.csv' 
    file_dir = os.path.dirname(os.path.abspath("_file_"))
    csv_folder = 'output/' + fecha_data.strftime('%Y') + "/" + fecha_data.strftime('%m')
    try:
        os.makedirs(os.path.join(file_dir, csv_folder))
    except FileExistsError:
        pass
    file_path = os.path.join(file_dir, csv_folder, nombre_archivo)
    df.to_csv(file_path, index=False)
    print('Archivo guardado', nombre_archivo)
    print("")



In [61]:
guarda_csv(df,"tottus")

Archivo guardado 20220906tottus.csv

